In [1]:
from lxml import etree
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import os,sys

### 接口

In [2]:
def voc2yolo():
    center_x = (xmin + (xmax-xmin)/2)/width
    center_y = (ymin + (ymax-ymin)/2)/height
    bdb_w = (xmax-xmin)/width
    bdb_h = (ymax-ymin)/height

In [3]:
def parse_xml(p):
    trees = etree.parse(str(p))
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    bboxes = []
    for obj in trees.findall('object'):
        bx = []
        bx.append(p)
        bx.append(int(width))
        bx.append(int(height))
        bdb = obj.find('bndbox')
        bx.append(obj.find('name').text)
        bx.append(int(bdb.find('xmin').text))
        bx.append(int(bdb.find('ymin').text))
        bx.append(int(bdb.find('xmax').text))
        bx.append(int(bdb.find('ymax').text))
        bboxes.append(bx)
    return bboxes

In [143]:
model2logic = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}
id9_map_logic = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装_短款
    3: 5,  # 裤子_长款
    4: 6,  # 裙子
    5: 7,  # 连体装
    6: 4,  # 上装_长款
    7: 5,  # 超短裤
    8: 4,  # 超短裙
}

id2chi9 = {
    0: '鞋子',
    1: '包包',
    2: '上装',
    3: '裤子',
    4: '半身裙',
    5: '连体装',
    6: '上装-长款',
    7: '超短裤',
    8: '超短裙',
}

id2eng9 = {
    0: 'shoe',  # 鞋子
    1: 'bag',  # 包包
    2: 'upper_short',  # 上装
    3: 'paints',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
    6: 'upper_long',
    7: 'paints_hot',
    8: 'miniskirt',
}

In [6]:
chi2id9 = dict(zip(id2chi9.values(), id2chi9.keys()))

### 1.Data Loader.

In [109]:
root = Path('/Users/chenyi/Desktop/data_aug/deploy_manual_sep/')
# !ls $root_xml
!ls $root/xml_rewrite/anomaly0826_xml

csv xml


In [111]:
root_img = root /'images/apparel/'
img_list = list(root_img.rglob('*.jpg'))
img_list += list(root_img.rglob('*.jpeg'))
img_list += list(root_img.rglob('*.png'))

img_df = pd.DataFrame()
img_df['path'] = list(img_list)
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)
len(img_list)

2081

In [112]:
root /'xml_rewrite/anomaly0826_xml/xml'

PosixPath('/Users/chenyi/Desktop/data_aug/deploy_manual_sep/xml_rewrite/anomaly0826_xml/xml')

In [126]:
# 核查xml是否有重复
root_xml = root /'xml_rewrite/'
# root_xml = root /'xml/'
xml_list = root_xml.rglob('*.xml')
xml_df = pd.DataFrame()
xml_df['path'] = list(xml_list)
xml_df['stem'] = xml_df['path'].apply(lambda x: Path(x).stem)
len(xml_df)

2069

#### csv读取和保存

In [150]:
csv_fname = 'anomoly2k_aug.csv'
root_csv = Path('/Users/chenyi/Desktop/data_aug/deploy_manual_sep/xml_rewrite/anomaly0826_xml/csv/')
csv_p = root_csv /csv_fname
# bboxes_df = pd.read_csv(root_csv /'gall7k_xml_20220715.csv')
# parts_df = pd.read_csv('../../data/xml_csv/gall7k_xml_20220714.csv')

In [151]:
# !mkdir -p $root_csv
# bboxes_df = bboxes_all_df
# bboxes_df.to_csv(str(csv_p), index=False)

### 解析XML

#### 解析XML

In [127]:
bboxes_list = []
for p in tqdm(xml_df['path'].values):
    try:
        row = parse_xml(p)
        bboxes_list += row
    except:
        print('empty xml.')
#     break

100%|██████████| 2069/2069 [00:01<00:00, 1749.01it/s]


In [128]:
bboxes_df = pd.DataFrame(bboxes_list)
name_stand = ['xml_path', 'width', 'height', 'labelimg', 'bx1', 'bx2','bx3', 'bx4']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                          bboxes_df.columns[7]:name_stand[7],
                         },
                          inplace=True)

In [129]:
bboxes_df['stem'] = bboxes_df['xml_path'].apply(lambda x: Path(x).stem)

##### 修正label

In [130]:
bboxes_df.value_counts('labelimg'),len(bboxes_df.value_counts('labelimg'))

(labelimg
 包包       914
 上装       592
 连体装      288
 裤子       131
 鞋子        96
 超短裙       76
 半身裙       66
 超短裤       54
 上装-长款     21
 dtype: int64,
 9)

In [25]:
label_name_chi  = '包'
name_new = '包包'
sample_df = bboxes_df[bboxes_df['labelimg']==label_name_chi]
bboxes_df.loc[bboxes_df[bboxes_df['labelimg']==label_name_chi].index, 'labelimg'] = name_new

In [131]:
bboxes_df['label9'] = bboxes_df['labelimg'].map(chi2id9)
bboxes_df['label_eng'] = bboxes_df['label9'].map(id2eng9)
bboxes_df['label6'] = bboxes_df['label9'].map(id9_map_model6)

#### 获取对应图片信息

##### 获取bbox对应的图片

In [132]:
# xml是否都有对应的图片
cross_df = img_df[img_df['stem'].isin(bboxes_df['stem'])]
len(cross_df)

2069

In [133]:
stem2path = dict(zip(cross_df['stem'].values, cross_df['path'].values))
bboxes_df['img_sp'] = bboxes_df['stem'].map(stem2path)
bboxes_clr_df = bboxes_df[~bboxes_df['img_sp'].isna()]
len(bboxes_df), len(bboxes_clr_df)

(2238, 2238)

##### 获取图片的宽和高

In [134]:
bboxes_df = bboxes_clr_df 

In [135]:
w_list = []
h_list = []
for ind,row in tqdm(bboxes_df.iterrows()):
    img_sp = row['img_sp']
    try:
        img = Image.open(img_sp)
        w, h = img.size
    except:
        try:
            img = cv2.imread(sample_p)
            w,h = img.shape[1], img.shape[0]
        except:
            w,h = None, None
    w_list.append(w)
    h_list.append(h)
    # break

bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

2238it [00:01, 1175.84it/s]


In [136]:
na_df = bboxes_df[bboxes_df['width'].isna()]
na_df

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,label9,label_eng,label6,img_sp


#### label处理

In [144]:
# bboxes_df['label9'] = bboxes_df['labelimg'].map(id2eng9)
# bboxes_df['label9'] = bboxes_df['labelimg'].map(eng2id9)
logic2model = dict(zip(model2logic.values(), model2logic.keys()))

In [145]:
# bboxes_df['label_model'] = bboxes_df['labelimg'].values
# bboxes_df['label_model'] = bboxes_df['label96'].values
# bboxes_df['label6'] = bboxes_df['label9'].map(id9_map_model6)
bboxes_df['label_model'] = bboxes_df['label6'].map(logic2model)
# bboxes_df.loc[bboxes_df['label_model']==6, 'label_model'] = 2
# bboxes_df.loc[bboxes_df['label_model']==7, 'label_model'] = 3
# bboxes_df.loc[bboxes_df['label_model']==8, 'label_model'] = 4

In [141]:
bboxes_df.columns

Index(['xml_path', 'width', 'height', 'labelimg', 'bx1', 'bx2', 'bx3', 'bx4',
       'stem', 'label9', 'label_eng', 'label6', 'img_sp'],
      dtype='object')

In [149]:
bboxes_df.value_counts('label9').sort_index()

label9
0     96
1    914
2    592
3    131
4     66
5    288
6     21
7     54
8     76
dtype: int64

In [147]:
bboxes_df.value_counts('label_model').sort_index()

label_model
0     96
1    914
2    689
3    185
4     66
5    288
dtype: int64

### one object.

In [ ]:
eng2num = {'shoe': 0, 
         'upperbody_short':2,
         'bag':1, 
         'lowbody_long':7, 
         'upperbody_long':6,
         'skirt':4, 
         'wholebody':5, 
         'lowbody_short':3}

In [ ]:
eng2num_hard = {'0_shoe': 0, 
                 '1_bag':1, 
         '2_upperbody_short':2,
         '3_lowbody_short':3, 
         '4_skirt':4, 
         '5_wholebody':5, 
         '6_upperbody_long':6,
         '7_lowbody_long':7}

In [ ]:
bboxes_df['cls_folder'] = bboxes_df['img_sp'].apply(lambda x: Path(x).parts[-2])

In [ ]:
# bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num)
bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num_hard)

In [ ]:
bboxes_eq_df = bboxes_df[bboxes_df['label_folder']==bboxes_df['label86']]

In [ ]:
bboxes_df['label_query'] = bboxes_df['label_folder']==bboxes_df['label86']

In [ ]:
len(bboxes_eq_df)

### 计算打标准确率

In [ ]:
parts_df = pd.read_csv('/nas/chenyi/datasets_nas/gallery2m_clred/gallery2m_7k20220715/csv/gall7k_xml_20220715.csv')

In [ ]:
parts_df.head(1)

In [ ]:
counter = parts_bboxes_df.value_counts('man').sort_index()
counter

In [ ]:
stem2part = get_list2dict(parts_df['stem'].values, parts_df['man'].values)

In [ ]:
parts_bboxes_df = parts_df[parts_df['stem'].isin(bboxes_df['stem'])]
len(parts_bboxes_df)

In [ ]:
bboxes_df['man'] = bboxes_df['stem'].map(stem2part)

In [ ]:
counter_gt = bboxes_df.value_counts('man').sort_index()
counter_gt

In [ ]:
def get_iou(BBGT, bb):
    # intersection
    ixmin = np.maximum(BBGT[:, 0], bb[0])
    iymin = np.maximum(BBGT[:, 1], bb[1])
    ixmax = np.minimum(BBGT[:, 2], bb[2])
    iymax = np.minimum(BBGT[:, 3], bb[3])
    iw = np.maximum(ixmax - ixmin + 1., 0.)
    ih = np.maximum(iymax - iymin + 1., 0.)
    inters = iw * ih        # union
    uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
           (BBGT[:, 2] - BBGT[:, 0] + 1.) *
           (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

    return inters / uni

In [ ]:
label_df1 = parts_bboxes_df[parts_bboxes_df['man']=='part5_img_xml']
gt_df1 = bboxes_df[bboxes_df['man']=='part5_img_xml']
parts_gp = dict(list(label_df1.groupby('stem')))
bboxes_gp = dict(list(gt_df1.groupby('stem')))

In [ ]:
tp_all, fp_all = 0, 0
stem_list = {}
for k in tqdm(bboxes_gp.keys()):
    item_part = parts_gp[k]
    item_bbox = bboxes_gp[k]
    bbx1 = item_part.loc[:, ['bx1', 'bx2', 'bx3', 'bx4', 'label9']].values
    bbx_gt = item_bbox.loc[:, ['bx1', 'bx2', 'bx3', 'bx4', 'label9']].values

    # bbxes_pred = bbx1
    # BBGT = bbx_gt
    # ovthresh = 0.90
    t, f = eval_one_img(bbx1, bbx_gt, ovthresh = 0.9)
    tp_all += t
    fp_all += f
    stem_list[k] = [t, f]

In [ ]:
# part1
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part2
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part3
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part4
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part5
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
fn_all = len(parts_bboxes_df) - tp_all

In [ ]:
tp_all / (tp_all + fp_all)

In [ ]:
tp_all / (tp_all + fn_all)

In [ ]:
def eval_one_img(bbxes_pred, BBGT, ovthresh = 0.5):
    # 按照置信度降序排序sorted_ind = np.argsort(-confidence)
    # BB = BB[sorted_ind, :]   # 预测框坐标image_ids = [image_ids[x] for x in sorted_ind] # 各个预测框的对应图片id# 便利预测框，并统计TPs和FPsnd = len(image_ids)

    # 获取每个类别的所有bboxes，包括gt和预测结果
    # 对于每张图，
    # 取检测的一个bbox bb
    # bb 和该图gt bbox bxes_gt计算 iou，去最大的iou

    ovmax = -np.inf
    tp, fp = 0, 0
    for bb in bbxes_pred:
        if len(BBGT) <= 0: 
            continue
        # 计算IoU
        cls_pred = bb[-1]
        BBGT_catg = BBGT[BBGT[:, -1]==cls_pred]
        if len(BBGT_catg) <=0:
            continue
        overlaps = get_iou(BBGT_catg, bb)
        ovmax = np.max(overlaps)   
        jmax = np.argmax(overlaps)    # 取最大的IoU的bbox索引
        if ovmax > ovthresh:  # 是否大于阈值
            tp += 1
#           删除已经匹配的检测框
            BBGT = np.concatenate([BBGT[~(BBGT[:, -1]==cls_pred)], 
                                   np.delete(BBGT_catg, jmax, axis = 0)], 
                                  axis = 0)
        else:
            fp += 1.
     
    return tp, fp

### XML REWRITER

In [38]:
#!/user/bin/env python3
# _*_ coding:utf-8 -*_
# using: 将数据写成xml标签

In [45]:
from xml.dom.minidom import Document
import os
import cv2
from pathlib import Path
import glob
from tqdm import tqdm

#### df转xml

In [101]:
xml_write_df = bboxes_df
xml_write_df['label_xml'] = xml_write_df['labelimg'].values
bboxes_gp = xml_write_df.groupby('stem')
dataset = 'deploy_manual_sep'
xml_write_df['xml_tp'] = xml_write_df['xml_path'].apply(lambda p: Path(str(p).replace('/deploy_manual_sep/xml/', '/deploy_manual_sep/xml_rewrite/')))

In [105]:
len(bboxes_df)

2238

In [ ]:
# class XMLWriter:
#     def __init__(self, data):
#         self.data = data
    
#     def write_xml_df(self):
#         xml_write_df = bboxes_df
#         label_name = 'labelimg'
#         xml_write_df['label_xml'] = xml_write_df['labelimg'].values
#         bboxes_gp = xml_write_df.groupby('stem')
#     dataset = 'deploy_manual_sep'
#     xml_write_df['xml_tp'] = xml_write_df['xml_path'].apply(lambda p: Path(str(p).replace('/deploy_manual_sep/xml/', '/deploy_manual_sep/xml_rewrite/')))

In [104]:
for item in tqdm(bboxes_gp):
    stem = item[0]
    rows = item[1]
    path, xml_tp, Pheight, Pwidth, Pdepth = rows.loc[:,['xml_path', 'xml_tp', 'width', 'height']].values[0, :].tolist() + [3]
    path, xml_tp = Path(path), Path(xml_tp)
    name_img = path.name
    bboxes = rows.loc[:,['label_xml', 'bx1', 'bx2', 'bx3', 'bx4']].values
    
    xmlBuilder = Document()
    annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
    xmlBuilder.appendChild(annotation)
    flag = 0
    for oneline in iter(bboxes):
        flag += 1
        folder = xmlBuilder.createElement("folder")  # folder标签
        folderContent = xmlBuilder.createTextNode(dataset)
        folder.appendChild(folderContent)
        annotation.appendChild(folder)
        if flag == 1:
            filename = xmlBuilder.createElement("filename")  # filename标签
            filenameContent = xmlBuilder.createTextNode(name[0:-4]+".jpg")
            filename.appendChild(filenameContent)
            annotation.appendChild(filename)

            size = xmlBuilder.createElement("size")  # size标签
            width = xmlBuilder.createElement("width")  # size子标签width
            widthContent = xmlBuilder.createTextNode(str(Pwidth))
            width.appendChild(widthContent)
            size.appendChild(width)
            height = xmlBuilder.createElement("height")  # size子标签height
            heightContent = xmlBuilder.createTextNode(str(Pheight))
            height.appendChild(heightContent)
            size.appendChild(height)
            depth = xmlBuilder.createElement("depth")  # size子标签depth
            depthContent = xmlBuilder.createTextNode(str(Pdepth))
            depth.appendChild(depthContent)
            size.appendChild(depth)
            annotation.appendChild(size)

        object = xmlBuilder.createElement("object")
        picname = xmlBuilder.createElement("name")
        nameContent = xmlBuilder.createTextNode(oneline[0])
        picname.appendChild(nameContent)
        object.appendChild(picname)
        pose = xmlBuilder.createElement("pose")
        poseContent = xmlBuilder.createTextNode("Unspecified")
        pose.appendChild(poseContent)
        object.appendChild(pose)
        truncated = xmlBuilder.createElement("truncated")
        truncatedContent = xmlBuilder.createTextNode("0")
        truncated.appendChild(truncatedContent)
        object.appendChild(truncated)
        difficult = xmlBuilder.createElement("difficult")
        difficultContent = xmlBuilder.createTextNode("0")
        difficult.appendChild(difficultContent)
        object.appendChild(difficult)
        bndbox = xmlBuilder.createElement("bndbox")
        xmin = xmlBuilder.createElement("xmin")
        mathData=int(oneline[1])
        xminContent = xmlBuilder.createTextNode(str(mathData))
        xmin.appendChild(xminContent)
        bndbox.appendChild(xmin)
        ymin = xmlBuilder.createElement("ymin")
        mathData = int(oneline[2])
        yminContent = xmlBuilder.createTextNode(str(mathData))
        ymin.appendChild(yminContent)
        bndbox.appendChild(ymin)
        xmax = xmlBuilder.createElement("xmax")
        mathData = int(oneline[3])
        xmaxContent = xmlBuilder.createTextNode(str(mathData))
        xmax.appendChild(xmaxContent)
        bndbox.appendChild(xmax)
        ymax = xmlBuilder.createElement("ymax")
        mathData = int(oneline[4])
        ymaxContent = xmlBuilder.createTextNode(str(mathData))
        ymax.appendChild(ymaxContent)
        bndbox.appendChild(ymax)
        object.appendChild(bndbox)

        annotation.appendChild(object)
    
    if xml_tp.is_file(): continue
    if not xml_tp.parent.is_dir(): xml_tp.parent.mkdir(parents=True)
    f = open(xml_tp, 'w')
    xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
    f.close()
#     break

100%|██████████| 2069/2069 [00:06<00:00, 296.38it/s]


In [ ]:
#!/user/bin/env python3
# _*_ coding:utf-8 -*_
# using: 将yolo txt label 转换成xml标签